In [8]:
import os
import time
import pickle
import numpy as np
import matplotlib, matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import torch
import itertools
from torch.autograd import Variable
import sklearn, sklearn.model_selection, sklearn.metrics
import numpy as np

from models.model_wrapper import MLP, GCN, SLR
from data import datasets
from data.graph_wrapper import GeneManiaGraph
from data.utils import record_result

In [2]:
dataset = datasets.TCGADataset()
dataset.df = dataset.df - dataset.df.mean(axis=0)

Torrent name: TCGA_tissue_ppi.hdf5, Size: 1748.32MB
Checking for pieces on disk: |██████████████████████████████████████████████████| 100.0% 
Found 1668 finished pieces out of 1668 total pieces.
Converting one-hot labels to integers


In [3]:
# Setup the results dictionary
filename = "experiments/results/fig-5.pkl"
try:
    results = pickle.load(open(filename, "rb"), encoding='latin1')
    print("Loaded Checkpointed Results")
except Exception as e:
    results = pd.DataFrame(columns=['auc', 'gene', 'model', 'num_genes', 'seed', 'train_size'])
    print("Created a New Results Dictionary")


Loaded Checkpointed Results


In [4]:
gene_graph = GeneManiaGraph()

In [5]:
search_num_genes=[16000]#[50, 100, 200, 300, 500, 1000, 2000, 4000, 8000, 16000]
test_size=100#1000
search_train_size=[50]
cuda=False
trials=1
search_genes = ["IL5"]#["RPL13", "CEBPD", "IL5", "PABPC3", "PSMB10", "RPL4", "RPL5", "RPS10", "RPS3", "S100A8", "S100A9", "TOP1", "C15orf40", "RNF138", "DLGAP2", "EVI2B", "ZFP82", "MYBL2", "PSMB1", "CISD1", "HLA-B", "SAA2", "IFIT1", "RPS3A", "TP53", "TNF", "EGFR"]
model_list = [              
               GCN(name="GCN_lay1_chan64_emb32_dropout_agg", cuda=cuda, num_layer=1, channels=64, embedding=32, pooling="hierarchy"),

              #GCN(name="GCN_lay20_chan32_emb32_dropout_pool", cuda=cuda, num_layer=4, channels=32, embedding=32, prepool_extralayers=5, pooling="hierarchy"),
#               MLP(name="MLP_lay2_chan512_dropout", cuda=cuda, dropout=True, num_layer=2, channels=512),
#               MLP(name="MLP_lay2_chan512", cuda=cuda, dropout=False, num_layer=2, channels=512),
#               SLR(name="SLR_lambda1_l11", cuda=cuda)
             ]

In [6]:
# Create the set of all experiment ids and see which are left to do
model_names = [model.name for model in model_list]
columns = ["gene", "model", "num_genes", "train_size", "seed"]
all_exp_ids = [x for x in itertools.product(search_genes, model_names, search_num_genes, search_train_size, range(trials))]
all_exp_ids = pd.DataFrame(all_exp_ids, columns=columns)
all_exp_ids.index = ["-".join(map(str, tup[1:])) for tup in all_exp_ids.itertuples(name=None)]
results_exp_ids = results[columns].copy()
results_exp_ids.index = ["-".join(map(str, tup[1:])) for tup in results_exp_ids.itertuples(name=None)]
intersection_ids = all_exp_ids.index.intersection(results_exp_ids.index)
todo = all_exp_ids.drop(intersection_ids).to_dict(orient="records")
print("todo: " + str(len(todo)))
print("done: " + str(len(results)))

todo: 1
done: 970


In [7]:
for row in todo:
    print(row)
    start_time = time.time()
    gene = row["gene"]
    model_name = row["model"]
    seed = row["seed"]
    num_genes = row["num_genes"]
    train_size = row["train_size"]

    model = [x for x in model_list if x.name == model_name][0]

    experiment = {
        "gene": gene,
        "model": model.name,
        "num_genes": num_genes,
        "train_size": train_size,
        "seed": seed,
    }

    dataset.labels = dataset.df[gene].where(dataset.df[gene] > 0).notnull().astype("int")
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(dataset.df, dataset.labels, stratify=dataset.labels, train_size=train_size, test_size=test_size, random_state=seed)

    neighbors = gene_graph.bfs_sample_neighbors(gene, num_genes)

    X_train = X_train[list(neighbors.nodes)].copy()
    X_test = X_test[list(neighbors.nodes)].copy()
    X_train[gene] = 1
    X_test[gene] = 1
    adj = np.asarray(nx.to_numpy_matrix(neighbors))
    model.fit(X_train, y_train, adj=adj)

    x_test = Variable(torch.FloatTensor(np.expand_dims(X_test.values, axis=2)), requires_grad=False).float()
    if cuda:
        x_test.cuda()
    y_hat = model.predict(x_test)[:,1].data.cpu().numpy()
    auc = sklearn.metrics.roc_auc_score(y_test, np.asarray(y_hat).flatten())

    model.best_model = None # cleanup
    experiment["auc"] = auc
    results = record_result(results, experiment, filename)
    print("time elapsed for genes: " +str(num_genes) + " : " + str(time.time() - start_time))


{'seed': 0, 'model': 'GCN_lay1_chan64_emb32_dropout_agg', 'gene': 'IL5', 'num_genes': 16000, 'train_size': 50}
before cached_cluster: 1547076305.211088
current_adj.sum(): 15397.205562488463
adj.sum(): 31113.9329677256
n_clusters.sum(): 8000
after cached_cluster: 1547076311.753118
> /Users/martinweiss/code/academic/conv-graph/models/graph_layers.py(61)__call__()
-> for j in range(adj.shape[0]):


(Pdb)  l


 56  	
 57  	        if self.type == 'max':
 58  	            xs = x.view(-1, x.size(-1))
 59  	            max_value = []
 60  	            import pdb; pdb.set_trace()
 61  ->	            for j in range(adj.shape[0]):
 62  	                neighbors = adj[:, j].nonzero()
 63  	                max_value.append(xs.t()[neighbors].max(dim=0)[0])
 64  	            import pdb; pdb.set_trace()
 65  	
 66  	            max_value = (x.view(-1, x.size(-1), 1) * adj).max(dim=1)[0]


(Pdb)  n


> /Users/martinweiss/code/academic/conv-graph/models/graph_layers.py(62)__call__()
-> neighbors = adj[:, j].nonzero()


...

 56  	
 57  	        if self.type == 'max':
 58  	            xs = x.view(-1, x.size(-1))
 59  	            max_value = []
 60  	            import pdb; pdb.set_trace()
 61  ->	            for j in range(adj.shape[0]):
 62  	                neighbors = adj[:, j].nonzero()
 63  	                max_value.append(xs.t()[neighbors].max(dim=0)[0])
 64  	            import pdb; pdb.set_trace()
 65  	
 66  	            max_value = (x.view(-1, x.size(-1), 1) * adj).max(dim=1)[0]


(Pdb)  c



Program interrupted. (Use 'cont' to resume).
--Call--
> /Users/martinweiss/.pyenv/versions/3.5.2/lib/python3.5/signal.py(45)signal()
-> @_wraps(_signal.signal)


(Pdb)  q


BdbQuit: 

In [7]:
6:25:40 start


In [13]:
norm_transform = D_inv.dot(adj).dot(D_inv)

16.7 s ± 1.77 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
asdf = sklearn.cluster.AgglomerativeClustering(n_clusters=4000, affinity='euclidean',
                                                                      memory='/tmp', connectivity=(adj > 0.).astype(int),
                                                                      compute_full_tree='auto', linkage='ward')

In [16]:
c = asdf.fit_predict(adj)

/Users/martinweiss/code/academic/conv-graph/venv/lib/python3.5/site-packages/sklearn/cluster/hierarchical.py:831: UserWarning: Persisting input arguments took 1.50s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **kwargs)


In [10]:
c = clustering.fit_predict((adj > 0.).astype(bool))

/Users/martinweiss/code/academic/conv-graph/venv/lib/python3.5/site-packages/sklearn/cluster/hierarchical.py:831: UserWarning: Persisting input arguments took 0.87s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **kwargs)


In [11]:
c = clustering.fit_predict((adj > 0.).astype(bool))

In [ ]:

for plot_gene in search_genes:

    %matplotlib inline
    plt.rcParams['figure.figsize'] = (7.5, 3.6)
    plot_train_size = 50

    subset = results[(results.train_size==plot_train_size) & 
                      (results.gene==plot_gene) & 
                      (results.num_genes!=400) &      
                      (results.num_genes> 0)]


    q = subset.groupby(['model','num_genes'])['auc']

    todo = list(subset["model"].unique())
    linestyles = ['-', '-', '--', '-.', ':']
    for ls, model in enumerate(sorted(todo)):
        index = list(q.mean()[model].index)
        mean = q.mean()[model]
        stderr = q.std()[model]/np.sqrt(q.count()[model])
        displayname = model.replace("CGN","GCN")
        displayname = displayname.replace("SLR", "SNLR")
        plt.errorbar(index, mean,label=displayname, xerr=0, yerr=stderr, ls=linestyles[ls])

    plt.title("Gene Inference " + plot_gene + " (train_size:" + str(plot_train_size) +")")
    plt.ylabel("AUC")
    plt.xlabel("Number of genes")
    plt.xscale("log")
    plt.xticks(sorted(subset["num_genes"].unique()))
    formatter = matplotlib.ticker.ScalarFormatter()
    plt.gca().xaxis.set_major_formatter(formatter)

    plt.legend();
    fd = len(list(gene_graph.nx_graph.neighbors(plot_gene)))
    print fd
    if fd > 50:
        plt.axvline(fd, ymin=0.4, ymax=1.0, c="black")
        c = plt.ylim()
        plt.text(fd*1.05,c[1]-((c[1]-c[0])*0.2),'First Degree',rotation=90)


    plt.savefig("experiments/results/sgi-" + plot_gene + "-" + "train" + str(plot_train_size) + ".png", bbox_inches='tight')
    plt.show()